In [1]:
import pandas as pd
import numpy as np

import sys
import os

# Añadir la carpeta utils al path
sys.path.append(os.path.abspath("../../utils"))

from functions import *

In [2]:
garmin_running = pd.read_csv("../../data/actividades_running/garmin_running.csv")

In [3]:
garmin_running.head()

,Activity Type,Date,Distance,Calories,Time,Avg HR,Max HR,Avg Run Cadence,Max Run Cadence,Avg Pace,...,Total Ascent,Total Descent,Avg Stride Length,Avg Vertical Ratio,Best Lap Time,Number of Laps,Moving Time,Elapsed Time,Min Elevation,Max Elevation
0,Running,2024-02-28 17:31:40,4.01,448,00:29:01,151,172,169,187,7:14,...,92,79,1.32,0.0,00:02.83,5,00:28:46,00:29:50,606,651
1,Running,2024-02-27 17:59:37,1.60,165,00:10:37,152,171,170,187,6:39,...,23,13,1.43,0.0,03:48.05,2,00:10:29,00:14:04,604,631
2,Running,2024-02-20 16:56:48,4.01,477,00:28:26,161,179,169,185,7:05,...,72,46,1.35,0.0,00:03.46,5,00:28:25,00:28:26,606,654
3,Running,2024-02-15 17:11:05,6.04,733,00:46:14,155,168,173,183,7:39,...,85,56,1.22,0.0,00:18.38,7,00:46:11,00:46:14,606,652
4,Running,2024-02-14 07:53:37,3.02,373,00:22:20,159,175,176,185,7:23,...,75,39,1.24,0.0,00:08.06,4,00:22:19,00:22:20,606,667


In [4]:
garmin_running["Activity Type"].unique()

array(['Running', 'Breathwork', 'Walking'], dtype=object)

Garmin coger Running

In [5]:
garmin_running = garmin_running[garmin_running["Activity Type"] == "Running"]

In [6]:
garmin_running["Activity Type"].unique()

array(['Running'], dtype=object)

In [7]:
garmin_running.info()

<class 'pandas.core.frame.DataFrame'>
Index: 312 entries, 0 to 313
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Activity Type       312 non-null    object 
 1   Date                312 non-null    object 
 2   Distance            312 non-null    object 
 3   Calories            312 non-null    object 
 4   Time                312 non-null    object 
 5   Avg HR              312 non-null    int64  
 6   Max HR              312 non-null    int64  
 7   Avg Run Cadence     312 non-null    object 
 8   Max Run Cadence     312 non-null    object 
 9   Avg Pace            312 non-null    object 
 10  Best Pace           312 non-null    object 
 11  Total Ascent        312 non-null    object 
 12  Total Descent       312 non-null    object 
 13  Avg Stride Length   312 non-null    float64
 14  Avg Vertical Ratio  312 non-null    float64
 15  Best Lap Time       312 non-null    object 
 16  Number of Lap

In [8]:
garmin_running.columns

Index(['Activity Type', 'Date', 'Distance', 'Calories', 'Time', 'Avg HR',
       'Max HR', 'Avg Run Cadence', 'Max Run Cadence', 'Avg Pace', 'Best Pace',
       'Total Ascent', 'Total Descent', 'Avg Stride Length',
       'Avg Vertical Ratio', 'Best Lap Time', 'Number of Laps', 'Moving Time',
       'Elapsed Time', 'Min Elevation', 'Max Elevation'],
      dtype='object')

In [9]:
# lista de columnas que queremos conservar
keep_cols = [
    'Date', 'Distance', 'Time', 'Avg HR',
    'Avg Run Cadence', 'Avg Pace',
    'Total Ascent', 'Avg Stride Length',
    'Number of Laps',
]

# por si alguna no existe, hacemos intersección
keep_cols = [c for c in keep_cols if c in garmin_running.columns]

# nos quedamos solo con esas
garmin_running = garmin_running[keep_cols].copy()

In [10]:
garmin_running.head()

,Date,Distance,Time,Avg HR,Avg Run Cadence,Avg Pace,Total Ascent,Avg Stride Length,Number of Laps
0,2024-02-28 17:31:40,4.01,00:29:01,151,169,7:14,92,1.32,5
1,2024-02-27 17:59:37,1.60,00:10:37,152,170,6:39,23,1.43,2
2,2024-02-20 16:56:48,4.01,00:28:26,161,169,7:05,72,1.35,5
3,2024-02-15 17:11:05,6.04,00:46:14,155,173,7:39,85,1.22,7
4,2024-02-14 07:53:37,3.02,00:22:20,159,176,7:23,75,1.24,4


Diferencia entre elapsed Time y Time

In [11]:
garmin_running.info()

<class 'pandas.core.frame.DataFrame'>
Index: 312 entries, 0 to 313
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Date               312 non-null    object 
 1   Distance           312 non-null    object 
 2   Time               312 non-null    object 
 3   Avg HR             312 non-null    int64  
 4   Avg Run Cadence    312 non-null    object 
 5   Avg Pace           312 non-null    object 
 6   Total Ascent       312 non-null    object 
 7   Avg Stride Length  312 non-null    float64
 8   Number of Laps     312 non-null    int64  
dtypes: float64(1), int64(2), object(6)
memory usage: 24.4+ KB


In [12]:
garmin_running["Date"] = pd.to_datetime(garmin_running["Date"])
garmin_running["Distance"] = pd.to_numeric(garmin_running["Distance"])
garmin_running["duration_s"] = garmin_running["Time"].apply(time_convert)
garmin_running["Avg Run Cadence"] = pd.to_numeric(garmin_running["Avg Run Cadence"])
garmin_running["pace_min_km"] = garmin_running["Avg Pace"].apply(pace_to_float)
garmin_running["pace_s_km"] = garmin_running["pace_min_km"] * 60
garmin_running["Total Ascent"] = pd.to_numeric(garmin_running["Total Ascent"])

In [13]:
garmin_running.drop(columns=["Time", "Avg Pace", "pace_min_km"], inplace=True)

In [14]:
garmin_running.info()

<class 'pandas.core.frame.DataFrame'>
Index: 312 entries, 0 to 313
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Date               312 non-null    datetime64[ns]
 1   Distance           312 non-null    float64       
 2   Avg HR             312 non-null    int64         
 3   Avg Run Cadence    312 non-null    int64         
 4   Total Ascent       312 non-null    int64         
 5   Avg Stride Length  312 non-null    float64       
 6   Number of Laps     312 non-null    int64         
 7   duration_s         312 non-null    float64       
 8   pace_s_km          312 non-null    float64       
dtypes: datetime64[ns](1), float64(4), int64(4)
memory usage: 24.4 KB


In [15]:
garmin_running.columns

Index(['Date', 'Distance', 'Avg HR', 'Avg Run Cadence', 'Total Ascent',
       'Avg Stride Length', 'Number of Laps', 'duration_s', 'pace_s_km'],
      dtype='object')

In [16]:
rename_map = {
    "Distance": "distance_km",
    "Avg HR": "avg_hr",
    "Avg Run Cadence": "avg_cadence",
    "Date": "date",
    "Total Ascent": "elevation_gain_m",
    "Avg Stride Length": "avg_stride_length_m",
    "Number of Laps": "laps"
}

In [17]:
garmin_running = garmin_running.rename(columns=rename_map)

In [18]:
garmin_running["runner"] = "garmin"

In [19]:
final_cols = [
    "runner", "date",
    "distance_km", "duration_s", "pace_s_km",
    "avg_hr", "avg_cadence",
    "avg_stride_length_m", "elevation_gain_m", "laps"
]

garmin_running = garmin_running[final_cols]

In [20]:
garmin_running.head()

,runner,date,distance_km,duration_s,pace_s_km,avg_hr,avg_cadence,avg_stride_length_m,elevation_gain_m,laps
0,garmin,2024-02-28 17:31:40,4.01,1741.0,434.0,151,169,1.32,92,5
1,garmin,2024-02-27 17:59:37,1.60,637.0,399.0,152,170,1.43,23,2
2,garmin,2024-02-20 16:56:48,4.01,1706.0,425.0,161,169,1.35,72,5
3,garmin,2024-02-15 17:11:05,6.04,2774.0,459.0,155,173,1.22,85,7
4,garmin,2024-02-14 07:53:37,3.02,1340.0,443.0,159,176,1.24,75,4


In [21]:
garmin_running.to_csv("../../data/actividades_running_limpias/garmin_running.csv", index=False)